# Реализация наивного Байесовского классификатора

In [1]:
import os

import numpy as np

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  # гарантируем воспроизводимость

run_env = os.getenv('RUN_ENV', 'COLLAB')
if run_env == 'COLLAB':
  from google.colab import drive
  ROOT_DIR = '/content/drive'
  drive.mount(ROOT_DIR)
  print('Google drive connected')
  DRIVE_DATA_DIR = 'ml_course_data'
  root_data_dir = os.path.join(ROOT_DIR, 'MyDrive', DRIVE_DATA_DIR)
else:
  root_data_dir = os.getenv('DATA_DIR', '/srv/data')

if not os.path.exists(root_data_dir):
  raise RuntimeError('Отсутствует директория с данными')
else:
  print('Содержимое директории %s: %s', root_data_dir, os.listdir(root_data_dir)[:5])

Содержимое директории %s: %s /Users/adzhumurat/PycharmProjects/ai_product_engineer/data ['client_segmentation.csv', 'messages.db', 'labeled_data_corpus.csv', 'content_description.csv', 'nltk_data']


[Dataset source](https://archive.ics.uci.edu/dataset/228/sms+spam+collection), но в директории с данными по курсу есть подготовленная версия

In [18]:
import pandas as pd

filename = os.path.join(root_data_dir, 'sms_spam_collection.csv')

df = pd.read_csv(filename).rename(columns={'label': 'class', 'message': 'sms_text'})

df.head(3)

,class,sms_text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...


Проверяем, сколько у нас всего объектов в датасете

In [19]:
num_objects, num_features = df.shape
print(num_objects, num_features)

5572 2


Целевая переменная (target) в столбце `class`

In [20]:
df['class'].head()

0     ham
1     ham
2    spam
3     ham
4     ham
Name: class, dtype: object

Демонстрация того, как получить булеву маску для датафрейма

In [40]:
SPAM_CLASS = 'spam'
NOT_SPAM_CLASS = 'ham'

df[df['class'] == SPAM_CLASS].head()

,class,sms_text,processed_text
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey there darling its been 3 weeks now...
8,spam,WINNER!! As a valued network customer you have...,winner as a valued network customer you have b...
9,spam,Had your mobile 11 months or more? U R entitle...,had your mobile 11 months or more u r entitled...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po...",six chances to win cash from 100 to 20000 poun...


Использование булевой маски для фильтрации датафрейма 

In [22]:
spam_sms_num = (df['class'] == SPAM_CLASS).sum()
notspam_sms_num = (df['class'] == NOT_SPAM_CLASS).sum()

print(f'spam sms: {spam_sms_num}, not spam sms {notspam_sms_num}')

spam sms: 747, not spam sms 4825


Пример - считаем вероятности классов

In [23]:
# априорная вероятность класса спам
p_spam = spam_sms_num / num_objects

# априорная вероятность класса не спам
p_notspam = notspam_sms_num / num_objects

print(f'{p_spam:.4f}, {p_notspam:.4f}')

0.1341, 0.8659


Пример обработки текстовой информации - приводим к нижнему регистру

In [24]:
test_word = 'Free'.lower()

test_word

'free'

In [25]:
sms_example = df['sms_text'].values[0]

sms_example

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [26]:
# априорная вероятность класса спам
p_spam = spam_sms_num / num_objects

# априорная вероятность класса не спам
p_notspam = notspam_sms_num / num_objects

print(f'{p_spam:.4f}, {p_notspam:.4f}')

0.1341, 0.8659


Пишем полезные сниппеты для трансформации текста

In [27]:
# удаляем знаки препинания
import string

print(string.punctuation)

sms_example = ''.join([
    char
    for char in sms_example
    if char not in string.punctuation
])

sms_example

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


'Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat'

In [28]:
# приводим слова к нижнему регистру

#.lower()
sms_example = ' '.join([
    word.lower()
    for word in sms_example.split(sep=' ')
])

sms_example

'go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat'

Объединяем сниппеты в функцию

In [29]:
import string

def text_preprocess(sms_text: str) -> str:
    """Преобразование текста для анализа"""
    text_no_punctuation = ''.join([
        char
        for char in sms_text
        if char not in string.punctuation
    ])
    text_lowercase = ' '.join([
        word.lower()
        for word in text_no_punctuation.split(sep=' ')
    ])
    
    return text_lowercase


sms_example = df['sms_text'].values[0]

print(text_preprocess(sms_example))

go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat


Трансформируем каждую строчку датафрейма

In [30]:
df = df.assign(
    processed_text=df['sms_text'].apply(text_preprocess)
)

df.head()

,class,sms_text,processed_text
0,ham,"Go until jurong point, crazy.. Available only ...",go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah i dont think he goes to usf he lives aroun...


Находим вероятность встретить слово в каждом из классов - это наша основная "фича" в наивном байесовском классификаторе

In [31]:
# вероятность встретить слово в спам смс

spam_test_word_entries = df[
    df['class'] == SPAM_CLASS
]['processed_text'].apply(
    lambda row: test_word in row
).sum()

# вероятность встретить слово в не-спам смс
notspam_test_word_entries = df[
    df['class'] == NOT_SPAM_CLASS
]['processed_text'].apply(
    lambda row: test_word in row
).sum()

print(f'P(word="{test_word}"|class=spam)={spam_test_word_entries/spam_sms_num:.4f}')
print(f'P(word="{test_word}"|class=not_spam)={notspam_test_word_entries/notspam_sms_num:.4f}')

P(word="free"|class=spam)=0.2664
P(word="free"|class=not_spam)=0.0137


### Вывод

слово "free" встречается в спам смс с вероятностью $26.6\%$, а в не-спаме с вероятностью $1.3\%$ - т.е. это слово является хорошим "маркером" спама

# Реализовать классификатор

Аналогично тому, как посчитали вероятности встретить слово `free` в каждом классе (спам/не спам) 
* в функции `fit()` подсчитать такие вероятности для каждого слова
* в функции `predict()` по формуле байеса (см. лекцию) вычислять вероятность принадлежности входного текста к каждому из классов

Результат предсказания - класс, вероятность принадлежности к которому больше

In [36]:
from typing import List
from copy import deepcopy
from collections import defaultdict

import numpy as np

"""имплементация наивного байесовского классификатора"""
class NaiveBayes:
    def __init__(self):
        
        self.labels = [NOT_SPAM_CLASS, SPAM_CLASS]
        self.class_labels_proba = None  # априорная вероятность класса
        self.prior_word_proba = None  # частоты фичей (токенов)
    
    def _set_labels_prior_proba(self, data: list, target: list):
        """Вычисление априорной вероятности классов"""
        class_labels_count = dict.fromkeys(self.labels, 0)

        num_objects = len(data)
        for label in target:
            try:
                class_labels_count[label] += 1
            except KeyError:
                print(f'Некорректное значение метки: {label}')

        class_labels_proba = {
            i: j / num_objects for i, j in class_labels_count.items()
        }
        self.class_labels_proba = class_labels_proba
    
    def _tokenize_text(self, text):
        tokens = []
        try:
            processed_text = text_preprocess(text)
            tokens = processed_text.split(' ')
        except TypeError:
            print(f'Ошибка при обработке текста sms: {text}')
        return tokens
    
    def _set_word_count(self, data, target):
        def_dict_obj = defaultdict(int)
        word_count_dict_by_class = dict.fromkeys(self.labels)
        for label in word_count_dict_by_class:
            word_count_dict_by_class[label] = deepcopy(def_dict_obj)
        word_count_dict_total = deepcopy(def_dict_obj)
        for label, text in zip(target, data):
                for token in self._tokenize_text(text):
                    word_count_dict_by_class[label][token] += 1
                    word_count_dict_total[token] += 1
        # переходим от счётчиков к частотам
        total_token_count = sum((count for token, count in word_count_dict_total.items()))
        print(f'total_token_count={total_token_count}')
        for label in word_count_dict_by_class:
            for token in word_count_dict_by_class[label]:
                word_count_dict_by_class[label][token] = word_count_dict_by_class[label][token] / total_token_count
        self.prior_word_proba = word_count_dict_by_class

    def fit(self, data: list, target: list):
        """Обучение статистик по датасету

        :param data: массив документов, каждый документ - объект типа str
        :param target: массив меток объектов
        :return:
        """
        if not isinstance(data, list):
            raise ValueError('Аргумент data должен иметь тип list')
        if not isinstance(target, list):
            raise ValueError('Аргумент target должен иметь тип list')
        print('Данные инициализированы!')
        self._set_labels_prior_proba(data, target)
        print(f'Априорные вероятности классов {self.class_labels_proba}')
        self._set_word_count(data, target)
        print('Обучили априорные вероятности слов')
        

    def _predict_proba(self, data: list) -> List[tuple]:
        """Предсказываем класс для текстовой смс

        :param data: массив документов, для каждого из которых нужно предсказать метку
        :return: вероятности для каждого из классов
        """
        prediction = []
        for obj in data:
            posterior_class_proba = defaultdict(lambda: 1)
            for token in self._tokenize_text(obj):
                for label in self.labels:
                    posterior_class_proba[label] *= self.prior_word_proba[label][token]
            # сохраняем для каждой метки класса - сколько меток, таков и размер uple
            prediction.append(
                tuple(
                    posterior_class_proba[label] for label in self.labels
                )
            )
        print(f'proba: {prediction}')
        return prediction
    
    def predict(self, data) -> List[str]:
        predict_labels = []
        for proba in self._predict_proba(data):
            predict_labels.append(self.labels[np.argmax(proba)])
        return predict_labels

naive_bayes = NaiveBayes()

In [37]:
naive_bayes.fit(
    data=df['sms_text'].values.tolist(),
    target=df['class'].tolist()
)

Данные инициализированы!
Априорные вероятности классов {'ham': 0.8659368269921034, 'spam': 0.13406317300789664}
total_token_count=87533
Обучили априорные вероятности слов


In [38]:
naive_bayes.prior_word_proba['ham']['thank'], naive_bayes.prior_word_proba['spam']['thank']

(0.0003084550969348703, 1.142426284943964e-05)

Предсказание метки класса

In [39]:
import numpy as np
# рандомный объект датасета

random_obj_ind = np.random.randint(low=0, high=num_objects, size=3)
random_obj_list = df['sms_text'].values[random_obj_ind].tolist()
random_target = df['class'][random_obj_ind].tolist()

print(random_obj_list)
naive_bayes.predict(
    random_obj_list
)

['Did he just say somebody is named tampa', 'Nt joking seriously i told', "Prabha..i'm soryda..realy..frm heart i'm sory"]
proba: [(1.9627768315243533e-26, 0.0), (1.710705384398593e-17, 0.0), (9.478535289379829e-20, 0.0)]


['ham', 'ham', 'ham']